# V1


In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import math

In [ ]:
A11 = np.array([[0.5, 0.2], [0.1, 0.4]])
A12 = np.array([[0.2], [0.1]])
A21 = np.array([[0.2, 0.2]])
A22 = np.array([[0.5]])

B1 = np.array([[0.8, 0.3], [1.0, 0.9]])
B2 = np.array([[0.3], [0.0]])

x10 = np.array([[1500], [1000]])
c10 = np.array([[500], [200]])

tau = 0.01

c2 = 400

def c1(c10, tau, t):
    return c10 * np.exp(tau * t)

In [ ]:
inv = np.linalg.inv(np.eye(1) - A22)

def c(A12, inv, c10, tau, t):
    return c1(c10, tau, t) - A12 @ inv * c2

c(A12, inv, c10, tau, 0)

A1 = A11 + np.dot(A12, inv @ A21)

A2 = A22 + np.dot(A21, np.dot(np.linalg.inv(B1 - A11), A12))

B = B1 + np.dot(B2, inv @ A21)

B_new = np.linalg.inv(B)

B_new[1, 0] + B_new[1, 1]
EA = np.eye(2) - A1
EA

from scipy.integrate import solve_ivp
import plotly.express as px
import pandas as pd

def system(t, z):
    x1, x2 = z

    dx1dt = B_new @ EA @ np.array([[x1], [x2]]) - B_new @ c(A12, inv, c10, tau, t)

    return dx1dt.ravel()

t = np.linspace(0, 1, 500)

x1_sol = solve_ivp(system, [0, 1], x10.ravel(), t_eval=t)
x2_sol = np.linalg.inv(np.eye(1) - A22)[0] * (0.2 * x1_sol.y[0] + 0.3 * x1_sol.y[1])

df = pd.DataFrame({
    'Time': np.tile(t, 3),
    'State': np.hstack((x1_sol.y[0], x1_sol.y[1], x2_sol)),
    'Variable': ['x11'] * len(t) + ['x12'] * len(t) + ['x2'] * len(t)
})
fig = px.line(df, x='Time', y='State', color='Variable',
              labels={'State': 'Value of State Variables'},
              title='General')
fig.show()

In [ ]:
from scipy.integrate import solve_ivp
import plotly.express as px
import pandas as pd

def system(t, z):
    x1, x2 = z

    dx1dt = B_new @ EA @ np.array([[x1], [x2]])

    return dx1dt.ravel()

t = np.linspace(0, 1, 500)

x1_sol = solve_ivp(system, [0, 1], x10.ravel(), t_eval=t)
x2_sol = np.linalg.inv(np.eye(1) - A22)[0] * (0.2 * x1_sol.y[0] + 0.3 * x1_sol.y[1])

df = pd.DataFrame({
    'Time': np.tile(t, 3),
    'State': np.hstack((x1_sol.y[0], x1_sol.y[1], x2_sol)),
    'Variable': ['x11'] * len(t) + ['x12'] * len(t) + ['x2'] * len(t)
})

fig = px.line(df, x='Time', y='State', color='Variable',
              labels={'State': 'Value of State Variables'},
              title='Closed')
fig.show()



In [ ]:
A11 = np.array([[0.5, 0.2], [0.1, 0.4]])
A12 = np.array([[0.2, 0.1]]).T
A21 = np.array([[0.2, 0.2]])
A22 = np.array([0.5])

B1 = np.array([[0.8, 0.3], [1.0, 0.9]])
B2 = np.array([[0.3, 0]]).T

x1_0 = np.array([[1500, 1000]]).T

def c1(t):
    return np.array([500, 200]) * np.exp(0.1*t)

c2 = 400

In [ ]:
A22_neg_inv = np.linalg.inv(np.eye(1) - A22)
A1 = A11 + np.dot(np.dot(A12, A22_neg_inv), A21)
B = B1 + np.dot(np.dot(B2, A22_neg_inv), A21)

def c(t):
    return c1(t) - np.dot(A22_neg_inv.T, A12.T).reshape(A12.shape[0]) * c2

A1, B

(array([[0.58, 0.28],
        [0.14, 0.44]]),
 array([[0.92, 0.42],
        [1.  , 0.9 ]]))

In [ ]:
B_inv = np.linalg.inv(B)
A1_neg_inv = np.linalg.inv(np.eye(A1.shape[0]) - A1)

def x_der(t, x):
    return np.dot(B_inv, x) - np.dot(np.dot(B_inv, A1), x) - np.dot(B_inv, c(t))

B_inv, A1_neg_inv

(array([[ 2.20588235, -1.02941176],
        [-2.45098039,  2.25490196]]),
 array([[2.85714286, 1.42857143],
        [0.71428571, 2.14285714]]))

In [ ]:
t = np.linspace(0, 5, 100)
x1 = solve_ivp(x_der, [0, 10], np.array([1500, 1000]), t_eval=t).y
x2 = np.dot(A22_neg_inv, (np.dot(A21, x1) - c2))



In [ ]:
def y_der(t, y):
    return np.dot(np.dot(B_inv, np.eye(A1.shape[0]) - A1), y)

In [ ]:
y1 = solve_ivp(y_der, [0, 100], [1500, 1000], t_eval=t).y
y2 = np.dot(A22_neg_inv, (np.dot(A21, y1)))


In [ ]:
eigen_values, vector = np.linalg.eig(np.dot(A1_neg_inv, B))
tech_growth = 1.0 / np.max(eigen_values)
print('Technology growth factor:')
print(tech_growth)

TIME_RANGE = 300

X1_tech_growth = [[1500, 1000]]
st = 1e-3
eps = 1e-3

while st < TIME_RANGE + 20:
    new_X1 = np.dot(X1_tech_growth[0], np.exp(tech_growth * st))
    X1_tech_growth.append(new_X1)
    st += eps

eigen_values, vector

Technology growth factor:
0.16849272549172106


(array([5.93497433, 0.35073996]),
 array([[ 0.79790805, -0.55699076],
        [ 0.60277919,  0.83051869]]))

In [ ]:
# Plot the final trajectory with technological growth
plt.plot(x1[0], x1[1], 'g', label='x1 trajectory')
plt.plot([x1_tg[0] for x1_tg in X1_tech_growth], [x1_tg[1] for x1_tg in X1_tech_growth], 'r', label='Tech growth trajectory')
plt.plot(y1[0], y1[1], 'b', label='y1 trajectory')
plt.axis([0, 4500, 0, 4500])
plt.legend()
plt.xlabel('x1_1 and y1_1')
plt.ylabel('x1_2 and y1_2')
plt.title('Phase trajectories')
plt.show()


In [ ]:
def c1(c10, tau, t):
    return c10 * np.exp(tau * t)

def c(A12, inv, c10, tau, t):
    return c1(c10, tau, t) - A12 @ inv * c2

X1_tech_growth = [[1500, 1000]]
eigenvals, eigenvecs = np.linalg.eig(np.linalg.inv(EA) @ B)
max_eigenval = np.argmax(eigenvals)

print(eigenvals)

tech_growth_rate = 1 / eigenvals[max_eigenval]
print(tech_growth_rate)

tech_growth_trajectory = eigenvecs[:, max_eigenval]
print(tech_growth_trajectory)


from scipy.integrate import solve_ivp
import pandas as pd
import matplotlib.pyplot as plt

def system(t, z):
    x1, x2 = z

    dx1dt = B_new @ EA @ np.array([[x1], [x2]])

    return dx1dt.ravel()

x_range = np.linspace(0, 10000, 20)
y_range = np.linspace(0, 10000, 20)
X, Y = np.meshgrid(x_range, y_range)
U, V = np.zeros(X.shape), np.zeros(Y.shape)
for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        rate = system(0, [X[i, j], Y[i, j]])
        U[i, j] = rate[0]
        V[i, j] = rate[1]

magnitude = np.sqrt(U**2 + V**2)
U_norm = U / magnitude
V_norm = V / magnitude

t = np.linspace(0, 10, 500)

x1_sol = solve_ivp(system, [0, 10], x10.ravel(), t_eval=t)

df = pd.DataFrame({
    'x11': x1_sol.y[0],
    'x12': x1_sol.y[1],
})

plt.quiver(X, Y, U_norm, V_norm, color='gray')

t_values = np.linspace(0, 7000, 500)
x_line = t_values * tech_growth_trajectory[0]
y_line = t_values * tech_growth_trajectory[1]
#print(x_line, y_line)
#plt.plot(x_line, y_line, label='Tech', color='red')
mask = (x_line >= 1600) | (y_line >= 1100)
filtered_x_line = x_line[mask]
filtered_y_line = y_line[mask]

plt.plot(filtered_x_line, filtered_y_line, label='Tech', color='red')


plt.plot(x1_sol.y[0], x1_sol.y[1], label="y1 trajectory")

def system(t, z):
    x1, x2 = z

    dx1dt = B_new @ EA @ np.array([[x1], [x2]]) - B_new @ c(A12, inv, c10, tau, t)

    return dx1dt.ravel()

x1_sol = solve_ivp(system, [0, 10], x10.ravel(), t_eval=t)

plt.plot(x1_sol.y[0], x1_sol.y[1], label="x1 trajectory", color='green')

plt.title('Phase Trajectories')
plt.xlabel('x1')
plt.ylabel('x2')
plt.legend()
plt.grid(True)
plt.xlim([0, 7000])
plt.ylim([0, 7000])

plt.gca().set_aspect('equal', adjustable='box')
